In [1]:
import pymol
import mbuild as mb
from mbuild.lib.recipes.polymer import Polymer
import warnings
warnings.filterwarnings('ignore')
import mdtraj as md
from openff.interchange import Interchange
from openmm import app
from openff.toolkit import ForceField, Molecule
from openff.toolkit.utils import get_data_file_path
import os

In [2]:
# Make output directory
output_dir = "output"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 1. Build PDB of helical peptide

## 1.1 PyMol Scripts
First we build the peptide sequence using PyMOLs built in peptide builder. Here we use the scripting version of PyMOL to build this peptide without interfacing with the PyMOL GUI.

In [3]:
peptide_sequence = "AAQAAAAQAA"
pymol.cmd.delete("all")
pymol.editor.fab(peptide_sequence, ss = 1, hydro=0)
pymol.editor.attach_amino_acid("last name C", 'nme', hydro=0)
pymol.editor.attach_amino_acid("first name N", 'ace', hydro=0)

# Change the naming of NME CH3 to C, so it is consistent with AMBER
pymol.cmd.alter("resn NME and name CH3", "name='C'")
pymol.cmd.set("pdb_conect_all", "off")
pymol.cmd.save(os.path.join(output_dir, "short_peptide.pdb"))

In [4]:
# Remove extra TER group after terminal alanine
with open(os.path.join(output_dir, "short_peptide.pdb"), 'r') as fn:
    lines = []
    for line in fn.readlines():
        if "TER" in line:
            continue
        else:
            lines.append(line)
with open(os.path.join(output_dir, "short_peptide.pdb"), 'w') as fn:
    fn.writelines(lines)

## 1.2 Adding Capping Residues

Next we'll use `mbuild` to add our capping residues to the generated PyMol generated peptide pdb. If the peptide generated above has capping residues that are available in PyMOL, these steps can be skipped.

In [ ]:
# Use mbuild to visualize componenets
peptide = mb.load(os.path.join(output_dir, "short_peptide.pdb"))
peptide
view = peptide.visualize(show_ports=True)
style = {
                "stick": {"radius": 0.2, "color": "grey"},
                "sphere": {"scale": 0.6, "color" : "black"},
    }
view.setStyle({'model': -1, 'serial':1},style)
view.setStyle({'model': -1, 'serial':129},style)

In [ ]:
succynl = mb.load("O=C(O)CCC(=O)O", smiles=True)
succynl.save("succ.pdb", overwrite=True)
succynl.remove(succynl[8])
view = succynl.visualize()

style = {
                "stick": {"radius": 0.2, "color": "grey"},
                "sphere": {"scale": 0.3, "color" : "black"},
    }
view.setStyle({'model': -1, 'serial':3},style)
#succynl = mb.load("O=C([O-])CCC(=O)O", smiles=True)
#view = succynl.visualize()
view.setStyle({'model': -1, 'serial':3},style)

In [ ]:
nh3 = mb.load("N", smiles=True)
nh3.save("nh3.pdb", overwrite=True)
view = nh3.visualize()
style = {
                "stick": {"radius": 0.2, "color": "grey"},
                "sphere": {"scale": 0.6, "color" : "black"},
    }
view.setStyle({'model': -1, 'serial':2},style)

In [ ]:
h2o = mb.load("O", smiles=True)
h2o.save("h2o.pdb", overwrite=True)
h2o.visualize()

Now that we have our peptide and cap residues, we will use `mbuild` Polymer object to combine them.

In [ ]:
chain = Polymer()

chain.add_monomer(compound = peptide, indices=[0, 128], separation=0.15, replace = False)
chain.add_end_groups(compound = succynl,
                     index = 2,
                     separation = 0.15,
                     label = "head",
                     duplicate = False,
                     orientation=[0.5,-0.5,0.5]
                    )

chain.add_end_groups(compound = nh3,
                     index = 2,
                     separation = 0.15,
                     label = "tail",
                     duplicate = False
                    )

chain.build(n=1)

In [ ]:
peptide = chain.children[0]

In [ ]:
chain.energy_minimize()
chain.visualize()

In [ ]:
# Set labels for capping residues
print(chain.children[1])
chain.children[1].name = "NTC"

print(chain.children[2])
chain.children[2].name = "CTC"

In [ ]:
# Verify we relabeled the non-standard capping residues
chain.labels['Compound'][0]

Now we save the peptide to file, ensuring amino acid residue names are written correctly. We also add the residues `NTC` and `CTC` for the N-terminal and C-terminal caps.

In [ ]:
chain.save("mbuild_small_peptide.pdb", overwrite = True, residues=["ALA", "LYS", "GLU", "TYR", "SER", "NTC", "CTC"])
smiles = chain.to_smiles()

# 2. AMBER Parameterization

Now we will use the AmberTools workflow to assign parameters for the peptide generated above.

In [5]:
import subprocess

peptide_pdb_fn = os.path.join(output_dir, "short_peptide.pdb")

## 2.1 Preparing the PDB file

We first run our PDB through `pdb4amber` which prepares the PDB file for parameter assignment using `tleap`. 

In [12]:
subprocess.Popen(["pdb4amber","-i", peptide_pdb_fn,"-o", os.path.join(output_dir, "peptide_amber.pdb",), "--reduce", "--add-missing-atoms"]).wait()


Summary of pdb4amber for: output/short_peptide.pdb

----------Chains
The following (original) chains have been found:


---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
None
-----------Non-standard-resnames


---------- Gaps (Renumbered Residues!)
gap of 4.090978 A between ACE 1 and ALA 3
gap of 3.227160 A between ALA 2 and GLN 4
gap of 3.229751 A between ALA 3 and ALA 5
gap of 3.226545 A between GLN 4 and ALA 6
gap of 3.227391 A between ALA 5 and ALA 7
gap of 3.226035 A between ALA 6 and ALA 8
gap of 3.227061 A between ALA 7 and GLN 9
gap of 3.229582 A between ALA 8 and ALA 10
gap of 3.227263 A between GLN 9 and ALA 11

---------- Missing heavy atom(s)

None


0

## 2.2 Computing Partial Charges with Antechamber

Next we use `antechamber` to assign partial charges for any custom residues. This requires having a `.cif` file of the custom residue used in your simulation.

```
$ antechamber -fi ccif -i CRO.cif -bk CRO -fo ac -o cro.ac -c bcc -at amber
```

## 2.3 Preparing residue library files

Here we prepare the custom residue library files to be included in the parameter assignment when we call `tleap`

## 2.4 Write `tleap` Input File 

We write an input file for `tleap` to assign parameters for the `peptide_amber.pdb` file we generated above. Additional custom residue prep and parameter files must be loaded using `loadAmberPrep` and `LoadAmberParams`.

In [13]:
amb_parm_file = os.path.join(output_dir,"peptide_amber.parm7")
amb_rst_file = os.path.join(output_dir, "peptide_amber.rst7")

lines = [
    "source leaprc.protein.ff14SB",
    "x = loadPDB " + os.path.join(output_dir,"peptide_amber.pdb"),
    "saveAmberParm x " + \
    amb_parm_file + " " + \
    amb_rst_file,
    "quit"
]

with open(os.path.join(output_dir, "tleap.in"), 'w') as fn:
    fn.write("\n".join(lines))

##  2.5 Run `tleap` for Parameter Assignment

In [14]:
subprocess.Popen(["tleap", "-f", os.path.join(output_dir, "tleap.in")]).wait()

-I: Adding /home/tfobe/anaconda3/envs/pymol_builder/dat/leap/prep to search path.
-I: Adding /home/tfobe/anaconda3/envs/pymol_builder/dat/leap/lib to search path.
-I: Adding /home/tfobe/anaconda3/envs/pymol_builder/dat/leap/parm to search path.
-I: Adding /home/tfobe/anaconda3/envs/pymol_builder/dat/leap/cmd to search path.
-f: Source output/tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./output/tleap.in
----- Source: /home/tfobe/anaconda3/envs/pymol_builder/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /home/tfobe/anaconda3/envs/pymol_builder/dat/leap/cmd/leaprc.protein.ff14SB done
Log file: ./leap.log
Loading parameters: /home/tfobe/anaconda3/envs/pymol_builder/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /home/tfobe/anaconda3/envs/pymol_builder/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain parameters
Load

0

## 3. Convert to GROMACS Parameter Files

Now we will use `InterMol` a program for converting parameters between different FF packages. Intermol generates a `.top` file that contains the parameters for the peptide

In [15]:
subprocess.Popen(["intermol-convert", "-f", "--amb_in", amb_parm_file, amb_rst_file, "--gromacs"]).wait()

/home/tfobe/Research/heteropolymer_simulations/examples/parameterize_peptide/output/peptide_amber.parm7 /home/tfobe/Research/heteropolymer_simulations/examples/parameterize_peptide/output/peptide_amber.rst7


{'gromacs': 'Converted'}


1

For some reason the output `peptide_amber_from_amber.gro` file is never formated correctly, so we will use `gmx editconf` to conver the PDB file output from `pdb4amber` to a GRO file.

In [16]:
subprocess.Popen(["gmx", "editconf", "-f", os.path.join(output_dir, "peptide_amber.pdb"), "-box", "10", "-o", os.path.join(output_dir, "peptide_amber.gro")])

<Popen: returncode: None args: ['gmx', 'editconf', '-f', 'output/peptide_amb...>

                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/heteropolymer_simulations/examples/parameterize_peptide
Command line:
  gmx editconf -f output/peptide_amber.pdb -box 10 -o output/peptide_amber.gro


GROMACS reminds you: "I'd Like Monday Mornings Better If They Started Later" (Garfield)



Note that major changes are planned in future for editconf, to improve usability and utility.
Read 126 atoms
No velocities found
    system size :  1.746  1.521  1.493 (nm)
    center      :  0.571  0.062 -0.468 (nm)
    box vectors :  0.000  0.000  0.000 (nm)
    box angles  :   0.00   0.00   0.00 (degrees)
    box volume  :   0.00               (nm^3)
    shift       :  4.429  4.938  5.468 (nm)
new center      :  5.000  5.000  5.000 (nm)
new box vectors : 10.000 10.000 10.000 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  :1000.00               (nm^3)
